### Import
---

In [1]:
import numpy as np
from note_include.elem.Ring import Ring # 00-Preliminaries 에서 작성한 Z_q[X]/(X^N+1) 에 대한 Class


---
# 02 - Ring-Learning with errors problem (RLWE)

이전에는 $\mathbb{Z}_q^n$ 상에서 동작하는 LWE 에 대해 알아보았다.

이번에는 우리가 앞서 공부하고 온 대수적 구조체(이하 $\mathcal{R}_Q = \mathbb{Z}_Q[X]/(X^N+1)$)를 활용한 RLWE 에 대해서 알아볼 것이다.

시작하기 앞서 RLWE 가 등장하게 된 배경부터 알아보자.

기존 LWE 의 경우에는 다음과 같은 한계점이 존재했다.

1. 동형 곱셈 연산의 정의가 어려움

2. 하나의 암호문에 하나의 메세지 (메모리)

3. Single Instruction Multiple Data (SIMD) 의 부재로 인한 연산의 효율성 저하

3번의 한계점과 같은 경우에는 FHEW 를 다루고 있는 지금은 크게 알아보지 않을 것이며 참고정도만 해두자.

---

그럼 이제 본격적으로 RLWE 에 대해서 알아보자. 간단하게 말하여 기존에 정수에서 정의되었던 LWE 를 다항식으로 확장한 문제라고 생각하면 편하다.

다음은 암호화의 정의를 나타내고 있다.

>$\textbf{Definition : RLWE encryption}$
>
>난수를 계수로 갖는 다항식 $\boldsymbol{a} \in \mathcal{R}_Q$ 와 비밀키 다항식 $\boldsymbol{s} \in \mathcal{R}_Q$ 그리고 작은 크기의 계수를 갖는 노이즈 다항식 $\boldsymbol{e} \in \mathcal{R}_Q$ 가 있다고 할 때 RLWE 암호화는 다음과 같이 정의된다:
>
>\begin{align*}
>    \textsf{RLWE.Enc}(\boldsymbol{m}, \boldsymbol{s}) = (\boldsymbol{a}, \boldsymbol{b}) \in \mathcal{R}_Q^2, \text{ where } \boldsymbol{b} =   \boldsymbol{a} \cdot \boldsymbol{s} + \boldsymbol{m} + \boldsymbol{e} \in \mathcal{R}_Q
>\end{align*}
>
>여기서 $\boldsymbol{m} \in \mathcal{R}_Q$ 은 메세지를 계수로 갖고있는 다항식을 의미한다.

참고할 것은 $(\cdot)$ 연산은 앞서 preliminaries 에서 다루었던 negacyclic convolution 을 나타낸다는 것이다.

코드를 통해 확인해보자.

In [2]:
# 정규분포(Gaussin distribution)를 따르는 난수 생성기
def discrete_gaussian(n, q, mean=0., std=3.2):
    coeffs = np.round(std * np.random.randn(n)) % q
    return np.array(coeffs, dtype = int)

# 균등분포(Uniform distribution)를 따르는 난수 생성기
def discrete_uniform(n, q, min=0., max=None):
    if max is None:
        max = q
    coeffs = np.random.randint(min, max, size=n)
    return np.array(coeffs, dtype = int)

In [3]:
def RLWE_Enc(m:Ring, s:Ring, std=3.2) -> tuple[Ring, Ring]:
    N = m.n
    Q = m.q
    a = Ring(N, Q, discrete_uniform(N, Q))       # Random Number
    e = Ring(N, Q, discrete_gaussian(N, Q, std=std)) # Noise

    b = a * s + m + e
    return (a, b)

In [28]:
N = 16
Q = 128

#   Ring(dimension, modulus, coeffs)
m = Ring(N, Q, [i for i in range(N)]) 
s = Ring(N, Q, discrete_gaussian(N, Q, std=3.2))

ct   = RLWE_Enc(m, s)
a, b = ct

print("Message             : ", m)
print("Secret key          : ", s)
print("a                   : ", a)
print("b (Encrypted value) : ", b)

Message             :  (0 + 1x + 2x^2 + 3x^3 + 4x^4 + 5x^5 + 6x^6 + 7x^7 + 8x^8 + 9x^9 + 10x^10 + 11x^11 + 12x^12 + 13x^13 + 14x^14 + 15x^15 | n=16, q=128)
Secret key          :  (126 + 0x + 8x^2 + 127x^3 + 127x^4 + 3x^5 + 4x^6 + 5x^7 + 0x^8 + 127x^9 + 1x^10 + 4x^11 + 126x^12 + 2x^13 + 123x^14 + 126x^15 | n=16, q=128)
a                   :  (29 + 62x + 72x^2 + 114x^3 + 110x^4 + 120x^5 + 92x^6 + 15x^7 + 39x^8 + 40x^9 + 69x^10 + 40x^11 + 109x^12 + 19x^13 + 10x^14 + 70x^15 | n=16, q=128)
b (Encrypted value) :  (76 + 55x + 12x^2 + 70x^3 + 94x^4 + 97x^5 + 117x^6 + 72x^7 + 0x^8 + 102x^9 + 79x^10 + 125x^11 + 100x^12 + 9x^13 + 124x^14 + 73x^15 | n=16, q=128)



---

실제로 보면 암호문이 무작위 난수처럼 보이고 있다는 것을 알 수 있다.

그럼 이번에는 다시 복호화를 정의해보자.

>$\textbf{Definition : RLWE decryption}$
>
>RLWE 암호문 $\textsf{ct} = (\boldsymbol{a}, \boldsymbol{b}) \in \mathcal{R}_Q^2$ 과 비밀키 $\boldsymbol{s} \in \mathcal{R}_Q$ 가 있다고 할 때 RLWE 복호화는 >다음과 같이 정의된다.
>
>\begin{align*}
>    \textsf{RLWE.Dec}(\textsf{ct}, \boldsymbol{s}) = \boldsymbol{b} - \boldsymbol{a} \cdot \boldsymbol{s} = \boldsymbol{m} + \boldsymbol{e} \approx \boldsymbol{m} \in \mathcal{R}_Q.
>\end{align*}

코드를 통해 확인해보자.

In [5]:
def RLWE_Dec(ct:tuple[Ring, Ring], s:Ring) -> Ring:
    a,b = ct
    msg = b - a * s

    return msg

In [36]:
N = 16
Q = 128

#   Ring(dimension, modulus, coeffs)
m = Ring(N, Q, [i * 5 for i in range(N)]) 
s = Ring(N, Q, discrete_gaussian(N, Q, std=3.2))

ct   = RLWE_Enc(m, s)
pt   = RLWE_Dec(ct, s)

print("Message         : ", m)
print("Decrypted value : ", pt)
print("Noise           : ", np.abs((m - pt).coeffs))

Message         :  (0 + 5x + 10x^2 + 15x^3 + 20x^4 + 25x^5 + 30x^6 + 35x^7 + 40x^8 + 45x^9 + 50x^10 + 55x^11 + 60x^12 + 65x^13 + 70x^14 + 75x^15 | n=16, q=128)
Decrypted value :  (1 + 2x + 12x^2 + 18x^3 + 19x^4 + 21x^5 + 30x^6 + 31x^7 + 39x^8 + 46x^9 + 51x^10 + 59x^11 + 58x^12 + 65x^13 + 71x^14 + 76x^15 | n=16, q=128)
Noise           :  [127   3 126 125   1   4   0   4   1 127 127 124   2   0 127 127]



---

메세지에 약간의 노이즈가 더해진 값을 반환한다는 것을 알 수 있다.

그럼 이번에는 암호문과 평문과의 덧셈과 곱셈을 정의한 뒤 살펴보도록 하자.

### 1. Plaintext Addition

>$\textbf{Definition : RLWE plaintext addition}$
>
>RLWE 암호문 $\textsf{ct} = (\boldsymbol{a}, \boldsymbol{b}) \in \mathcal{R}_Q^2$ 과 평문 $\boldsymbol{t} \in \mathcal{R}_Q$ 가 있다고 할 때 RLWE plaintext  addition 은 다음과 같이 정의할 수 있다:
>
>\begin{align*}
>    \textsf{RLWE.Add}(\textsf{ct}, \boldsymbol{t}) = (\boldsymbol{a}, \boldsymbol{b} + \boldsymbol{t}) \in \mathcal{R}_Q^2
>\end{align*}

다음은 이에 대한 correctness 를 알아보자.

>$\textbf{Correctness : RLWE plaintext addition}$
>
>$\textsf{ct}_\text{add} = \textsf{RLWE.Add}(\textsf{ct}, \boldsymbol{t})$ 라 할 때 다음의 수식이 성립한다.
>
>\begin{align*}
>    \textsf{RLWE.Dec}(\textsf{ct}_\text{add}) &= (\boldsymbol{b} + \boldsymbol{t}) - \boldsymbol{a} \cdot \boldsymbol{s} \\
>                                              &= \boldsymbol{m} + \boldsymbol{t} + \boldsymbol{e} \approx \boldsymbol{m} + \boldsymbol{t}
>\end{align*}

코드를 통해 확인해보자.

In [7]:
def RLWE_Add_Plaintext(ct:tuple[Ring, Ring], t:Ring):
    a,b = ct
    return (a, b + t)

In [8]:
N = 16
Q = 128

m = Ring(N, Q, [i * 5 for i in range(N)])
t = Ring(N, Q, [i for i in range(N)])
s = Ring(N, Q, discrete_gaussian(N, Q, std=3.2))

ct     = RLWE_Enc(m, s)
ct_add = RLWE_Add_Plaintext(ct, t) 
pt_add = RLWE_Dec(ct_add, s)

print("Ideal Result    : ", m + t)
print("Decrypted value : ", pt_add)
print("Noise           : ", np.abs((m + t - pt_add).coeffs))

Ideal Result    :  (0 + 6x + 12x^2 + 18x^3 + 24x^4 + 30x^5 + 36x^6 + 42x^7 + 48x^8 + 54x^9 + 60x^10 + 66x^11 + 72x^12 + 78x^13 + 84x^14 + 90x^15 | n=16, q=128)
Decrypted value :  (2 + 125x + 8x^2 + 12x^3 + 31x^4 + 29x^5 + 35x^6 + 40x^7 + 48x^8 + 53x^9 + 53x^10 + 70x^11 + 72x^12 + 77x^13 + 87x^14 + 96x^15 | n=16, q=128)
Noise           :  [126   9   4   6 121   1   1   2   0   1   7 124   0   1 125 122]



---

실제로 거의 유사한 값이 나오는 것을 알 수 있다. 이어서 평문 곱셈을 정의해보자.

### 2. Plaintext Multiplication

>$\textbf{Definition : RLWE plaintext multiplication}$
>
>RLWE 암호문 $\textsf{ct} = (\boldsymbol{a}, \boldsymbol{b}) \in \mathcal{R}_Q^2$ 과 평문 $\boldsymbol{t} \in \mathcal{R}_Q$ 가 있다고 할 때 RLWE plaintext  multiplication 은 다음과 같이 정의할 수 있다:
>
>\begin{align*}
>    \textsf{RLWE.Mul}(\textsf{ct}, \boldsymbol{t}) = (\boldsymbol{a} \cdot \boldsymbol{t}, \boldsymbol{b} \cdot \boldsymbol{t}) \in \mathcal{R}_Q^2
>\end{align*}

다음은 이에 대한 correctness 를 알아보자.

>$\textbf{Correctness : RLWE plaintext addition}$
>
>$\textsf{ct}_\text{mul} = \textsf{RLWE.Mul}(\textsf{ct}, \boldsymbol{t})$ 라 할 때 다음의 수식이 성립한다.
>
>\begin{align*}
>    \textsf{RLWE.Dec}(\textsf{ct}_\text{add}) &= (\boldsymbol{b} \cdot \boldsymbol{t}) - (\boldsymbol{a} \cdot \boldsymbol{t}) \cdot \boldsymbol{s} \\
>                                              &= \boldsymbol{a} \cdot \boldsymbol{t} \cdot \boldsymbol{s} +  \boldsymbol{t} \cdot \boldsymbol{m} + \boldsymbol{t} \cdot \boldsymbol{e} - \boldsymbol{a} \cdot \boldsymbol{t} \cdot \boldsymbol{s} \\
>                                              &= \boldsymbol{m} \cdot \boldsymbol{t} + \boldsymbol{e} \cdot \boldsymbol{t} \approx \boldsymbol{m} \cdot \boldsymbol{t}
>\end{align*}

코드를 통해 확인해보자.

In [9]:
def RLWE_Mul_Plaintext(ct:tuple[Ring, Ring], t:Ring) -> tuple[Ring, Ring]:
    a,b = ct
    return (a*t, b*t)

In [10]:
N = 16
Q = 128

m = Ring(N, Q, [i * 5 for i in range(N)])
t = Ring(N, Q, [1 for i in range(N)])
s = Ring(N, Q, discrete_gaussian(N, Q, std=3.2))

ct     = RLWE_Enc(m, s)
ct_mul = RLWE_Mul_Plaintext(ct, t) 
pt_mul = RLWE_Dec(ct_mul, s)

print("Ideal Result    : ", m * t)
print("Decrypted value : ", pt_mul)
print("Noise           : ", np.abs((m * t - pt_mul).coeffs))

Ideal Result    :  (40 + 50x + 70x^2 + 100x^3 + 12x^4 + 62x^5 + 122x^6 + 64x^7 + 16x^8 + 106x^9 + 78x^10 + 60x^11 + 52x^12 + 54x^13 + 66x^14 + 88x^15 | n=16, q=128)
Decrypted value :  (49 + 53x + 79x^2 + 111x^3 + 17x^4 + 67x^5 + 7x^6 + 63x^7 + 15x^8 + 105x^9 + 73x^10 + 59x^11 + 51x^12 + 51x^13 + 61x^14 + 85x^15 | n=16, q=128)
Noise           :  [119 125 119 117 123 123 115   1   1   1   5   1   1   3   5   3]



---

얼추 비슷한 값이 나오는 것을 확인할 수 있으며, 일반적으로 곱셈 연산이 덧셈 연산에 비해서 더 큰 노이즈를 발생시킨다는 것을 참고하기를 바란다.

다시 강조하지만 여기서 사용되는 $(\cdot)$ 연산은 negacyclic convolution 이다.

이제 이어서 동형암호의 꽃인 동형연산을 정의해보자. 시작은 동형덧셈이다.

### 3. Ciphertext Addition (Homomorphic Addition)

>$\textbf{Definition : RLWE ciphertext addition}$
>
>두 RLWE 암호문 $\textsf{ct}_1 = (\boldsymbol{a}_1, \boldsymbol{b}_1) \in \mathcal{R}_Q^2$ 과 $\textsf{ct}_2 = (\boldsymbol{a}_2, \boldsymbol{b}_2) \in \mathcal{R}_Q^2$ 가 있다고 할 때, RLWE ciphertext addition 은 다음과 같이 정의된다:
>
>\begin{align*}
>    \textsf{RLWE.Add}(\textsf{ct}_1, \textsf{ct}_2) = (\boldsymbol{a}_1 + \boldsymbol{a}_2, \boldsymbol{b}_1 + \boldsymbol{b}_2) \in \mathcal{R}_Q^2.
>\end{align*}

다음으로 correctness 를 확인해보자.

>$\textbf{Correctness : RLWE ciphertext addition}$
>
>$\textsf{ct}_\text{add} = \textsf{RLWE.Add}(\textsf{ct}_1, \textsf{ct}_2)$ 라 할 때 다음의 수식이 성립한다.
>
>\begin{align*}
>
>    \textsf{RLWE.Dec}(\textsf{ct}_\text{add}) &= \boldsymbol{b}_\text{add} - \boldsymbol{a}_\text{add} \cdot \boldsymbol{s} \\
>                                              &= (\boldsymbol{b}_1 + \boldsymbol{b}_2) - (\boldsymbol{a}_1 + \boldsymbol{a}_2) \cdot \boldsymbol{s} \\
>                                              &= (\boldsymbol{b}_1 - \boldsymbol{a}_1 \cdot \boldsymbol{s}) +
>                                                 (\boldsymbol{b}_2 - \boldsymbol{a}_2 \cdot \boldsymbol{s}) \\
>                                              &= \boldsymbol{m}_1 + \boldsymbol{e}_1 + \boldsymbol{m}_2 + \boldsymbol{e}_2
>                                                 \approx \boldsymbol{m}_1 + \boldsymbol{m}_2
>
>\end{align*}

코드를 통해 확인해보자.

In [11]:
def RLWE_Add_Ciphertext(ct1:tuple[Ring, Ring], ct2:tuple[Ring, Ring]) -> tuple[Ring, Ring]:
    a1, b1 = ct1
    a2, b2 = ct2

    return (a1 + a2, b1 + b2)

In [12]:
N = 16
Q = 128

m1 = Ring(N, Q, [i for i in range(N)])
m2 = Ring(N, Q, [N-i for i in range(N)])
s  = Ring(N, Q, discrete_gaussian(N, Q, std=3.2))

ct1     = RLWE_Enc(m1, s)
ct2     = RLWE_Enc(m2, s)

ct_add = RLWE_Add_Ciphertext(ct1, ct2) 
pt_add = RLWE_Dec(ct_add, s)

print("Ideal Result    : ", m1 + m2)
print("Decrypted value : ", pt_add)
print("Noise           : ", np.abs(((m1 + m2) - pt_add).coeffs))

Ideal Result    :  (16 + 16x + 16x^2 + 16x^3 + 16x^4 + 16x^5 + 16x^6 + 16x^7 + 16x^8 + 16x^9 + 16x^10 + 16x^11 + 16x^12 + 16x^13 + 16x^14 + 16x^15 | n=16, q=128)
Decrypted value :  (17 + 17x + 21x^2 + 13x^3 + 17x^4 + 16x^5 + 14x^6 + 8x^7 + 12x^8 + 8x^9 + 4x^10 + 12x^11 + 16x^12 + 13x^13 + 17x^14 + 20x^15 | n=16, q=128)
Noise           :  [127 127 123   3 127   0   2   8   4   8  12   4   0   3 127 124]



---

### 4. Ciphertext Multiplication (Homomorphic Multiplication)

마지막으로 동형 곱셈에 대해서 확인해보자. 동형 곱셈의 경우에는 약간 복잡하다.

설명의 편의를 동형 곱셈을 통해 얻고 싶어하는 메세지가 무엇인지 우선 파악하고, 이를 맞추어서 연산을 정의해보자.

동형 곱셈의 결과를 $\textsf{ct}_\text{mul}$ 이라고 해보자. 우리가 동형 곱셈의 대한 결과로 얻고 싶은 값은 다음과 같은 수식으로 표현할 수 있다:

\begin{align*}
    \textsf{RLWE.Dec}(\textsf{ct}_\text{mul}) &= \textsf{RLWE.Dec}(\textsf{ct}_1) \cdot \textsf{RLWE.Dec}(\textsf{ct}_2) \\
                                              &= (\boldsymbol{b}_1 - \boldsymbol{a}_1 \cdot \boldsymbol{s}) \cdot
                                                 (\boldsymbol{b}_2 - \boldsymbol{a}_2 \cdot \boldsymbol{s}) \\
                                              &= \boldsymbol{b}_1 \boldsymbol{b}_2 - 
                                                 \boldsymbol{a}_1 \boldsymbol{b}_2 \cdot \boldsymbol{s} -                       
                                                 \boldsymbol{a}_2 \boldsymbol{b}_1 \cdot \boldsymbol{s} +                      
                                                 \boldsymbol{a}_1 \boldsymbol{a}_2 \cdot \boldsymbol{s}^2 \\
                                              &= \boldsymbol{b}_1 \boldsymbol{b}_2 -
                                                (\boldsymbol{a}_1 \boldsymbol{b}_2 + \boldsymbol{a}_2 \boldsymbol{b}_1) 
                                                 \cdot \boldsymbol{s} +
                                                 \boldsymbol{a}_1 \boldsymbol{a}_2 \cdot \boldsymbol{s}^2 \\
                                              &= (\boldsymbol{m}_1 + \boldsymbol{e}_1) \cdot (\boldsymbol{m}_2 + \boldsymbol{e}_2) \\
                                              &= \boldsymbol{m}_1 \cdot \boldsymbol{m}_2 + (\boldsymbol{m}_1 \cdot \boldsymbol{e}_2) +
                                                 (\boldsymbol{m}_2 \cdot \boldsymbol{e}_1) + (\boldsymbol{e}_1 \cdot \boldsymbol{e}_2) \\
                                              &\approx \boldsymbol{m}_1 \cdot \boldsymbol{m}_2
\end{align*}

여기서 알 수 있는 점은 동형 곱셈을 정의하기 위해서 동형 곱셈 뿐만 아니라 곱셈의 결과에 대한 복호화 알고리즘의 정의 또한 바꾸어야 한다는 것이다.

그럼 다음과 같이 동형 곱셈을 정의해보자.

>$\textbf{Definition : RLWE ciphertext multiplication}$
>
>두 RLWE 암호문 $\textsf{ct}_1 = (\boldsymbol{a}_1, \boldsymbol{b}_1) \in \mathcal{R}_Q^2$ 과 
>$\textsf{ct}_2 = (\boldsymbol{a}_2, \boldsymbol{b}_2) \in \mathcal{R}_Q^2$ 가 있다고 할 때, RLWE ciphertext multiplication 은 다음과 같이 정의된다:
>
>\begin{align*}
>    \textsf{RLWE.Mul}(\textsf{ct}_1, \textsf{ct}_2) = (
>        \boldsymbol{a}_1 \boldsymbol{b}_2 + \boldsymbol{a}_2 \boldsymbol{b}_1, \boldsymbol{a}_1 \boldsymbol{a}_2, \boldsymbol{b}_1 \boldsymbol{b}_2
>    ) = (\boldsymbol{d}_0, \boldsymbol{d}_1, \boldsymbol{d}_2) \in \mathcal{R}_Q^3
>\end{align*}

여기서 참고할 점은 암호문의 차원이 기존 $\mathcal{R}_Q^2$ 에서 $\mathcal{R}_Q^3$ 으로 늘어났다는 것이다.

늘어난 차원에 따라 RLWE 암호문의 복호화 또한 다시 정의해 주어야 한다.

>$\textbf{Definition : Multiplied RLWE ciphertext decryption}$
>
>$\textsf{ct}_\text{mul} = \textsf{RLWE.Mul}(\textsf{ct}_1, \textsf{ct}_2) \in \mathcal{R}_Q^3$ 이라 할 때 이에 대한 복호화는 다음과 같이 정의된다:
>
>\begin{align*}
>    \textsf{RLWE.Dec}(\textsf{ct}_\text{mul}) = 
>    \boldsymbol{d}_2 - \boldsymbol{d}_0 \cdot \boldsymbol{s} + \boldsymbol{d}_1 \cdot \boldsymbol{s}^2
>\end{align*}

코드를 통해 확인해보자.

In [13]:
def RLWE_Mul_Ciphertext(ct1:tuple[Ring, Ring], ct2:tuple[Ring, Ring]) -> tuple[Ring, Ring, Ring]:
    a1, b1 = ct1
    a2, b2 = ct2

    d0 = a1 * b2 + a2 * b1
    d1 = a1 * a2
    d2 = b1 * b2

    return [d0, d1, d2]

def RLWE_Dec_Mul(ct:tuple[Ring, Ring, Ring], s:Ring) -> Ring:
    d0, d1, d2 = ct
    msg = d2 - (d0*s) + (d1*s*s)

    return msg

In [14]:
N = 16
Q = 128

m1 = Ring(N, Q, [i for i in range(N)])
m2 = Ring(N, Q, [i for i in range(N)])
s  = Ring(N, Q, discrete_gaussian(N, Q, std=3.2))

ct1     = RLWE_Enc(m1, s)
ct2     = RLWE_Enc(m2, s)

ct_mul = RLWE_Mul_Ciphertext(ct1, ct2) 
pt_mul = RLWE_Dec_Mul(ct_mul, s)

print("Ideal Result    : ", m1 * m2)
print("Decrypted value : ", pt_mul)
print("Noise           : ", np.abs(((m1 * m2) - pt_mul).coeffs))

Ideal Result    :  (88 + 112x + 26x^2 + 88x^3 + 44x^4 + 24x^5 + 30x^6 + 64x^7 + 0x^8 + 96x^9 + 98x^10 + 8x^11 + 84x^12 + 72x^13 + 102x^14 + 48x^15 | n=16, q=128)
Decrypted value :  (113 + 99x + 46x^2 + 21x^3 + 10x^4 + 62x^5 + 117x^6 + 33x^7 + 127x^8 + 103x^9 + 30x^10 + 54x^11 + 23x^12 + 46x^13 + 78x^14 + 39x^15 | n=16, q=128)
Noise           :  [103  13 108  67  34  90  41  31   1 121  68  82  61  26  24   9]


노이즈가 상당히 크다는 것을 알 수 있다. 잘 동작함을 확인하기 위해 잠시 노이즈를 제거하고 결과를 확인해보자.

In [15]:
N = 16
Q = 128

m1 = Ring(N, Q, [i for i in range(N)])
m2 = Ring(N, Q, [i for i in range(N)])
s  = Ring(N, Q, discrete_gaussian(N, Q, std=3.2))

ct1     = RLWE_Enc(m1, s, std=0.) # Noiseless encryption
ct2     = RLWE_Enc(m2, s, std=0.) # Noiseless encryption

ct_mul = RLWE_Mul_Ciphertext(ct1, ct2) 
pt_mul = RLWE_Dec_Mul(ct_mul, s)

print("Ideal Result    : ", m1 * m2)
print("Decrypted value : ", pt_mul)
print("Noise           : ", np.abs(((m1 * m2) - pt_mul).coeffs))

Ideal Result    :  (88 + 112x + 26x^2 + 88x^3 + 44x^4 + 24x^5 + 30x^6 + 64x^7 + 0x^8 + 96x^9 + 98x^10 + 8x^11 + 84x^12 + 72x^13 + 102x^14 + 48x^15 | n=16, q=128)
Decrypted value :  (88 + 112x + 26x^2 + 88x^3 + 44x^4 + 24x^5 + 30x^6 + 64x^7 + 0x^8 + 96x^9 + 98x^10 + 8x^11 + 84x^12 + 72x^13 + 102x^14 + 48x^15 | n=16, q=128)
Noise           :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]



---

## Summary

RLWE 암호화, 복호화 알고리즘을 정의하고 RLWE 암호문에 대해서 수행할 수 있는 4가지 연산을 정의해 보았다.

1. Plaintext Addition

2. Plaintext Multiplication

3. Ciphertext Addition

4. Ciphertext Multiplication

한 가지 중요한 점은 Ciphertext Multiplication 의 경우 노이즈의 발생이 매우 커서 메세지를 확인하기가 어려웠다는 사실이다.

또한 암호문의 차원이 증가하는 문제점도 발생하는데, 이는 곱셈을 수행하면 수행할수록 계속 증가하게 된다.

이렇듯 동형암호에서는 비효율적인 곱셈을 최적화하기 위해 다양한 테크닉을 사용한다.

크게 다음 두 가지 측면을 개선하기 위해 사용한다고 보면 된다.

* 곱셈에서 발생하는 노이즈를 줄이기 위함

* 곱셈을 수행할 경우 늘어나는 차원을 방지하기 위함

FHEW 에서는 RLWE 의 변형인 RLWE' 과 RGSW 를 정의하여 곱셈을 수행하고, 다른 동형암호 스킴인 CKKS/BFV/BGV 의 경우에는 Rescaling 과 Relinearization 을 활용하여 곱셈을 개선한다.

Rescaling 과 Relinearization 은 다루지 않을 것이며 추후 기회가 된다면 소개하도록 하겠다.

---

## Code Implementation

`note_include/elem/RLWE.py` 를 확인하면 위에서 정의한 연산들이 있는 RLWE 구현을 확인할 수 있다. (동형곱셈은 구현하지 않았다.)